### Example of Model Composition

<img src="images/PatternsMLProduction.png" width="70%" height="40%">

#### Install HuggingFace Transformers and Torch modules

In [1]:
!pip install transformers torch

  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached torch-1.11.0-cp38-none-macosx_10_9_x86_64.whl (129.9 MB)
  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
  Using cached sacremoses-0.0.49-py3-none-any.whl (895 kB)
  Using cached tokenizers-0.12.1-cp38-cp38-macosx_10_11_x86_64.whl (3.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 820.6 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 KB 411.3 kB/s eta 0:00:0000:0100:01


In [2]:
from transformers import TranslationPipeline, TextClassificationPipeline
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification
import torch
import requests
from ray import serve

Example tweetes. These could come live from a Tweeter
handle using Twitter APIs. 

In [1]:
TWEETS = ["Tonight on my walk, I got mad because mom wouldn't let me play with this dog. We stared at each other...he never blinked!",
          "Sometimes. when i am bored. i will stare at nothing. and try to convince the human. that there is a ghost",
          "You little dog shit, you peed and pooed on my new carpet. Bad dog!",
          "I would completely believe you. Dogs and little children - very innocent and open to seeing such things",
          "You've got too much time on your paws. Go check on the skittle. under the, fridge",
          "You sneaky little devil, I can't live without you!!!",
          "It's true what they say about dogs: they are you BEST BUDDY, no matter what!",
          "This dog is way dope, just can't enough of her",
          "This dog is way cool, just can't enough of her",
          ]

In [2]:
# function to fetch a tweet; this could very well be live 
# tweets coming from Twitter API for a user or #hashtag
def fetch_tweet_text(i):
    text = TWEETS[i]
    return text

In [6]:
# Function to analyse the tweet using a pretrained Transformer
# from HuggingFace
@serve.deployment(num_replicas=1)
def sentiment_model(text: str):
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
    pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, task="sentiment-analysis")

    return pipeline(text)[0]['label'], pipeline(text)[0]['score']

In [7]:
# Function to translate a tweet from English --> French 
# using a pretrained Transformer from HuggingFace
@serve.deployment(num_replicas=2)
def translate_model(text: str):
    tokenizer = AutoTokenizer.from_pretrained("t5-small")
    model = AutoModelWithLMHead.from_pretrained("t5-small")
    use_gpu = 0 if torch.cuda.is_available() else -1
    pipeline = TranslationPipeline(model, tokenizer, task="translation_en_to_fr", device=use_gpu)

    return pipeline(text)[0]['translation_text']

In [8]:
# A composed class is deployed with both sentiment analysis and translations models'
# Serve Handles initialized in the constructor
@serve.deployment(route_prefix="/composed", num_replicas=2)
class ComposedModel:
    def __init__(self):
        # fetch and initialize deployment handles
        self.translate_model = translate_model.get_handle(sync=False)
        self.sentiment_model = sentiment_model.get_handle(sync=False)

    async def __call__(self, starlette_request):
        data = starlette_request.query_params['data']

        sentiment, score = await(await self.sentiment_model.remote(data))
        trans_text = await(await self.translate_model.remote(data))

        return {'Sentiment': sentiment, 'score': score, 'Translated Text': trans_text}

In [9]:
serve.start()

2022-04-26 16:12:50,963	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265
(ServeController pid=61718) 2022-04-26 16:12:53,596	INFO checkpoint_path.py:15 -- Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=61718) 2022-04-26 16:12:53,704	INFO http_state.py:106 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:WJOxXp:SERVE_PROXY_ACTOR-node:127.0.0.1-0' on node 'node:127.0.0.1-0' listening on '127.0.0.1:8000'
2022-04-26 16:12:54,550	INFO api.py:794 -- Started Serve instance in namespace 'serve'.


(HTTPProxyActor pid=61720) INFO:     Started server process [61720]


In [10]:
sentiment_model.deploy()
translate_model.deploy()
ComposedModel.deploy()

2022-04-26 16:13:06,178	INFO api.py:615 -- Updating deployment 'sentiment_model'. component=serve deployment=sentiment_model
(ServeController pid=61718) 2022-04-26 16:13:06,198	INFO deployment_state.py:1210 -- Adding 1 replicas to deployment 'sentiment_model'. component=serve deployment=sentiment_model
2022-04-26 16:13:09,194	INFO api.py:630 -- Deployment 'sentiment_model' is ready at `http://127.0.0.1:8000/sentiment_model`. component=serve deployment=sentiment_model
2022-04-26 16:13:09,204	INFO api.py:615 -- Updating deployment 'translate_model'. component=serve deployment=translate_model
(ServeController pid=61718) 2022-04-26 16:13:09,308	INFO deployment_state.py:1210 -- Adding 2 replicas to deployment 'translate_model'. component=serve deployment=translate_model
2022-04-26 16:13:12,220	INFO api.py:630 -- Deployment 'translate_model' is ready at `http://127.0.0.1:8000/translate_model`. component=serve deployment=translate_model
2022-04-26 16:13:12,228	INFO api.py:615 -- Updating depl

In [11]:
for i in range(len(TWEETS)):
    tweet = fetch_tweet_text(i)
    print(F"Sending tweet request... : {tweet}")
    resp = requests.get("http://127.0.0.1:8000/composed", params={'data': tweet})
    print(resp.json())

Sending tweet request... : Tonight on my walk, I got mad because mom wouldn't let me play with this dog. We stared at each other...he never blinked!


(translate_model pid=61724) /usr/local/anaconda3/envs/ray-core-tutorial/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:907: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
(translate_model pid=61724)   warnings.warn(


{'Sentiment': 'POSITIVE', 'score': 0.965121328830719, 'Translated Text': "Ce soir, j'ai été fou parce que ma mère ne me laisse pas jouer avec ce chien."}
Sending tweet request... : Sometimes. when i am bored. i will stare at nothing. and try to convince the human. that there is a ghost
{'Sentiment': 'NEGATIVE', 'score': 0.99788898229599, 'Translated Text': "Parfois. quand j'ennuie. je ne regarderai rien. et essayerai de convaincre l'homme."}
Sending tweet request... : You little dog shit, you peed and pooed on my new carpet. Bad dog!


(translate_model pid=61725) /usr/local/anaconda3/envs/ray-core-tutorial/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:907: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
(translate_model pid=61725)   warnings.warn(


{'Sentiment': 'NEGATIVE', 'score': 0.9984055161476135, 'Translated Text': "Je n'ai pas eu l'impression d'avoir eu l'impression d'avoir eu l'impression d'avoir eu l'impression d'avoir eu l'impression d'avoir eu l'impression d'avoir eu l'impression d'avoir eu l'impression d'avoir eu l'impression d'avoir eu l'impression"}
Sending tweet request... : I would completely believe you. Dogs and little children - very innocent and open to seeing such things
{'Sentiment': 'POSITIVE', 'score': 0.9997748732566833, 'Translated Text': 'Je vous croyais tout à fait: chiens et petits enfants - très innocents et ouverts à ce genre de choses'}
Sending tweet request... : You've got too much time on your paws. Go check on the skittle. under the, fridge
{'Sentiment': 'NEGATIVE', 'score': 0.9995866417884827, 'Translated Text': 'Vous avez trop de temps sur vos pattes.'}
Sending tweet request... : You sneaky little devil, I can't live without you!!!
{'Sentiment': 'POSITIVE', 'score': 0.9949393272399902, 'Transl

In [12]:
serve.shutdown()

(ServeController pid=61718) 2022-04-26 16:19:06,268	INFO deployment_state.py:1236 -- Removing 1 replicas from deployment 'sentiment_model'. component=serve deployment=sentiment_model
(ServeController pid=61718) 2022-04-26 16:19:06,272	INFO deployment_state.py:1236 -- Removing 2 replicas from deployment 'translate_model'. component=serve deployment=translate_model
(ServeController pid=61718) 2022-04-26 16:19:06,276	INFO deployment_state.py:1236 -- Removing 2 replicas from deployment 'ComposedModel'. component=serve deployment=ComposedModel
